In [1]:
import numpy as np 
import pandas as pd
import re
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import train_test_split
from scipy import special
from tqdm import tqdm
import warnings
from sklearn.metrics import roc_auc_score, f1_score, classification_report, accuracy_score, roc_curve
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from lightgbm import LGBMClassifier, LGBMRegressor
import lightgbm as lgb
from sklearn.metrics import roc_auc_score, classification_report
from sklearn_genetic import GAFeatureSelectionCV
from sklearn_genetic.callbacks import ProgressBar, ConsecutiveStopping, TensorBoard
from sklearn.feature_selection import RFE

warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:20,.2f}'.format)

In [2]:
X_train = pd.read_csv('/Users/sakshamjain/Desktop/Projects/JAIN-WIN/X_train_corr.csv')
y_train = pd.read_csv('/Users/sakshamjain/Desktop/Projects/JAIN-WIN/y_train_large.csv').squeeze()
X_test = pd.read_csv('/Users/sakshamjain/Desktop/Projects/JAIN-WIN/X_test_corr.csv')
y_test = pd.read_csv('/Users/sakshamjain/Desktop/Projects/JAIN-WIN/y_test_large.csv').squeeze()

In [3]:
X_train.columns = X_train.columns.str.replace(' ', '_')
X_test.columns = X_test.columns.str.replace(' ', '_')

In [4]:
X_train.head()

,age,bmi,elective_surgery,gender,height,icu_id,pre_icu_los_days,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_invasive_max,d1_diasbp_invasive_min,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_invasive_max,d1_mbp_invasive_min,d1_mbp_max,d1_mbp_min,d1_mbp_noninvasive_max,d1_mbp_noninvasive_min,d1_resprate_max,d1_resprate_min,d1_spo2_max,d1_spo2_min,d1_sysbp_invasive_max,d1_sysbp_invasive_min,d1_sysbp_max,d1_sysbp_min,d1_sysbp_noninvasive_max,d1_sysbp_noninvasive_min,d1_temp_max,d1_temp_min,h1_diasbp_invasive_max,h1_diasbp_invasive_min,h1_diasbp_max,h1_diasbp_min,h1_diasbp_noninvasive_max,h1_diasbp_noninvasive_min,h1_heartrate_max,h1_heartrate_min,h1_mbp_invasive_max,h1_mbp_invasive_min,h1_mbp_max,h1_mbp_min,h1_mbp_noninvasive_max,h1_mbp_noninvasive_min,h1_resprate_max,h1_resprate_min,h1_spo2_max,h1_spo2_min,h1_sysbp_invasive_max,h1_sysbp_invasive_min,h1_sysbp_max,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,h1_temp_max,h1_temp_min,d1_albumin_max,d1_albumin_min,d1_bilirubin_max,d1_bilirubin_min,d1_bun_max,d1_bun_min,d1_calcium_max,d1_calcium_min,d1_creatinine_max,d1_creatinine_min,d1_glucose_max,d1_glucose_min,d1_hco3_max,d1_hco3_min,d1_hemaglobin_max,d1_hemaglobin_min,d1_hematocrit_max,d1_hematocrit_min,d1_inr_max,d1_inr_min,d1_lactate_max,d1_lactate_min,d1_platelets_max,d1_platelets_min,d1_potassium_max,d1_potassium_min,d1_sodium_max,d1_sodium_min,d1_wbc_max,d1_wbc_min,h1_albumin_max,h1_albumin_min,h1_bilirubin_max,h1_bilirubin_min,h1_bun_max,h1_bun_min,h1_calcium_max,h1_calcium_min,h1_creatinine_max,h1_creatinine_min,h1_glucose_max,h1_glucose_min,h1_hco3_max,h1_hco3_min,h1_hemaglobin_max,h1_hemaglobin_min,h1_hematocrit_max,h1_hematocrit_min,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,icu_id_counts,age_counts,nan_counts,sq_age,sq_bmi,bmi_age,weight_age,comorbidity_score,diabete_risk,gcs_sum,apache_2_diagnosis_type,bmi_type,height_type,weight_type,age_type,gcs_sum_type,apache_3j_diagnosis_split1,apache_2_diagnosis_split1,profile,diff_bmi,bilirubin_d1_h1_max_eq,bilirubin_d1_h1_min_eq,bilirubin_h1_value_range,bilirubin_h1_zero_range,bilirubin_h1_mean,bilirubin_h1_std,bilirubin_h1_max_apache_3j_diagnosis_mean,bilirubin_h1_max_apache_3j_diagnosis_diff,bilirubin_h1_max_apache_3j_diagnosis_std,bilirubin_h1_max_apache_3j_diagnosis_norm_std,bilirubin_h1_max_apache_3j_diagnosis_rank,bilirubin_h1_max_apache_3j_diagnosis_count,bilirubin_h1_max_apache_3j_diagnosis_norm_rank,bilirubin_h1_max_apache_3j_diagnosis_skew,bilirubin_h1_min_apache_3j_diagnosis_mean,bilirubin_h1_min_apache_3j_diagnosis_diff,bilirubin_h1_min_apache_3j_diagnosis_std,bilirubin_h1_min_apache_3j_diagnosis_norm_std,bilirubin_h1_min_apache_3j_diagnosis_rank,bilirubin_h1_min_apache_3j_diagnosis_norm_rank,bilirubin_h1_min_apache_3j_diagnosis_skew,bilirubin_h1_max_profile_mean,bilirubin_h1_max_profile_diff,bilirubin_h1_max_profile_std,bilirubin_h1_max_profile_norm_std,bilirubin_h1_max_profile_rank,bilirubin_h1_max_profile_count,bilirubin_h1_max_profile_norm_ran

In [5]:
print("Training LightGBM model...")
model = lgb.LGBMClassifier( random_state=69, n_jobs=-1, force_col_wise=True ,verbose=0)
# Train the initial LightGBM model
model.fit(X_train, y_train)

y_pred_proba = model.predict_proba(X_test)[:, 1]
y_pred = model.predict(X_test)

# Calculate classification metrics
auc_roc = roc_auc_score(y_test, y_pred_proba)
print("AUC-ROC:", auc_roc)
print("Classification Report:\n", classification_report(y_test, y_pred))

Training LightGBM model...
AUC-ROC: 0.8720636771207815
Classification Report:
               precision    recall  f1-score   support

         0.0       0.87      0.94      0.90     30574
         1.0       0.69      0.50      0.58      8474

    accuracy                           0.84     39048
   macro avg       0.78      0.72      0.74     39048
weighted avg       0.83      0.84      0.83     39048



In [6]:
feature_importance = model.feature_importances_
feature_names = X_train.columns

# Create a DataFrame for importance
importance_df = pd.DataFrame({'feature': feature_names, 'importance': feature_importance})

# Sort by importance
importance_df = importance_df.sort_values(by='importance', ascending=False).reset_index(drop=True)

# Calculate cumulative importance
importance_df['cumulative_importance'] = importance_df['importance'].cumsum() / importance_df['importance'].sum()

# Select features contributing to 95% of cumulative importance
selected_features = importance_df[importance_df['cumulative_importance'] <= 0.95]['feature']

# Filter the train and test sets for selected features
X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]

# Retrain the model using the selected features
model_selected = lgb.LGBMClassifier(random_state=69, n_jobs=-1, force_col_wise=True)

# Train the model again on the selected features
model_selected.fit(X_train_selected, y_train)

# Make predictions and evaluate the model
y_pred_selected = model_selected.predict_proba(X_test_selected)[:, 1]
auc_roc_selected = roc_auc_score(y_test, y_pred_selected)

print(f"AUC-ROC on the selected features: {auc_roc_selected}")

AUC-ROC on the selected features: 0.8723564971241229


In [7]:
X_train=X_train_selected.copy()
X_test=X_test_selected.copy()

In [8]:
X_train

glucose_d1_value_range                  age  glucose_d1_min_icu_id_mean           arf_apache           age_counts  glucose_d1_max_icu_id_norm_rank  glucose_apache_d1_max_ratio  d1_hemaglobin_max_bmi_ratio       d1_glucose_max         diabete_risk  spo2_h1_max_apache_3j_diagnosis_skew  spo2_h1_max_icu_id_mean  glucose_d1_max_apache_3j_diagnosis_norm_rank        sodium_d1_std  glucose_d1_max_icu_id_diff  glucose_d1_min_icu_id_std  lactate_d1_min_icu_id_skew  creatinine_d1_min_profile_mean  glucose_h1_max_icu_id_skew  temp_h1_min_icu_id_mean  arterial_ph_h1_min_icu_id_mean  d1_wbc_max_bmi_ratio  d1_glucose_min_bmi_ratio  d1_heartrate_max_bmi_ratio  arterial_po2_h1_min_apache_3j_diagnosis_mean  temp_d1_min_icu_id_mean     sodium_d1_h1_std               icu_id     pre_icu_los_days  glucose_d1_min_icu_id_norm_rank  glucose_d1_max_apache_3j_diagnosis_diff  h1_diasbp_max_bmi_ratio  glucose_d1_min_apache_3j_diagnosis_mean  spo2_d1_max_icu_id_skew           bun_apache  \
0                       -0.33                -0.06                       -0.24                -0.17                 0.58                            -0.18                         0.36                        -0.93                -0.49                 0.43                                 -0.27                    -0.42                                         -0.49                -0.53                        0.52                       0.00                       -0.53                            1.78                        0.02                     0.62                           -1.01                  0.01                     -1.15                       -0.88                                         -0.33                     1.21                -0.77                 0.81                -0.48                            -0.40                                     0.67                    -0.53                                    -0.29                    -0.03                -0.58   
1                        1.68                -2.54                        1.49                -0.17                -2.05                             1.35                        -0.47                        -0.44                 1.82                 0.69                                 -2.59                    -1.90                                         -0.39                -0.16                       -1.67                       2.16                       -1.53                           -1.49                        0.51                     0.19                            0.05                 -0.25                      0.11                        0.07                                         -0.92                    -0.76                -0.95                -0.54                -0.63                             0.69                                     0.79                     0.33                                     0.22                     0.29                -0.68   
2                        0.25                -0.06                        0.53                -0.17                -3.84                            -0.19                         2.30                         0.22                -0.38                -2.60                                 -0.12                     0.36                                         -0.36                -0.53                        0.48                       0.74                        1.27                           -0.67                        0.53                     1.01                            0.38                  0.21                     -0.73                        1.42                                         -0.02                     1.15                 0.96                 1.46                 0.10                            -1.53                                     0.56                     0.14                                     0.31                     0.57                 0.09   
3                       -0.72                 0.67                        1

In [22]:
def find_dataframes():
    # This will check for variables that are instances of pd.DataFrame in the global scope
    return {name: obj for name, obj in globals().items() if isinstance(obj, pd.DataFrame)}

def print_memory_usage_of_dataframes():
    dataframes = find_dataframes()
    total_memory = 0
    print("Memory usage of dataframes (in GB):")
    for name, df in dataframes.items():
        mem_usage = df.memory_usage(deep=True).sum() / 1024 ** 3  # Convert bytes to gigabytes
        total_memory += mem_usage
        print(f"{name}: {mem_usage:.6f} GB")
    print(f"Total memory used by dataframes: {total_memory:.6f} GB")

print_memory_usage_of_dataframes()

Memory usage of dataframes (in GB):
X_train: 0.433169 GB
X_test: 0.185650 GB
Total memory used by dataframes: 0.618819 GB


In [12]:
model = lgb.LGBMClassifier( random_state=69, n_jobs=-1, force_col_wise=True ,verbose=1)


rfe = RFE(estimator=model, n_features_to_select=639, step=0.05)
print("Fitting RFE...")
rfe.fit(X_train, y_train)
print("RFE fitting completed.\n")

# Get the selected features
selected_features_rfe = X_train.columns[rfe.support_].tolist()
print(f"Selected {len(selected_features_rfe)} features after RFE: {selected_features_rfe}\n")

X_train_rfe = X_train[selected_features_rfe]
X_test_rfe = X_test[selected_features_rfe]

# Train LightGBM on RFE-selected Features
print("Training LightGBM model on RFE-selected features...")
model_rfe = lgb.LGBMClassifier( random_state=69, n_jobs=-1, force_col_wise=True )
model_rfe.fit(X_train_rfe, y_train)
print("Model trained on RFE-selected features.\n")

# Predictions and Probabilities
y_pred_rfe = model_rfe.predict(X_test_rfe)
y_proba_rfe = model_rfe.predict_proba(X_test_rfe)[:, 1]

# Metrics
print("Classification Report for RFE-selected Features:")
print(classification_report(y_test, y_pred_rfe))
auc_rfe = roc_auc_score(y_test, y_proba_rfe)
print(f"AUC-ROC for RFE-selected Features: {auc_rfe:.8f}\n")

Fitting RFE...
[LightGBM] [Info] Number of positive: 19677, number of negative: 71432
[LightGBM] [Info] Total Bins 311277
[LightGBM] [Info] Number of data points in the train set: 91109, number of used features: 1277
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.215972 -> initscore=-1.289296
[LightGBM] [Info] Start training from score -1.289296
[LightGBM] [Info] Number of positive: 19677, number of negative: 71432
[LightGBM] [Info] Total Bins 295793
[LightGBM] [Info] Number of data points in the train set: 91109, number of used features: 1214
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.215972 -> initscore=-1.289296
[LightGBM] [Info] Start training from score -1.289296
[LightGBM] [Info] Number of positive: 19677, number of negative: 71432
[LightGBM] [Info] Total Bins 280191
[LightGBM] [Info] Number of data points in the train set: 91109, number of used features: 1151
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.215972 -> initscore=-1.289296
[LightGBM] [Info] Start trainin

In [13]:
X_train=X_train_rfe.copy()
X_test=X_test_rfe.copy()

In [14]:
del X_train_rfe,X_test_rfe

In [15]:
model = lgb.LGBMClassifier( random_state=69, n_jobs=-1, force_col_wise=True,verbose=0)

In [23]:
selector=GAFeatureSelectionCV(
    estimator=model,
    max_features=320,
    scoring="roc_auc",
    n_jobs=-1,
    verbose=True,
    generations=10,
    population_size=100
)


progress_bar = ProgressBar()
tensor= TensorBoard()
stopper= ConsecutiveStopping(generations=5, metric="fitness_max")
callbacks = [progress_bar, tensor, stopper]

In [24]:
selector.fit(X_train, y_train,callbacks=callbacks)

  0%|          | 0/11 [00:00<?, ?it/s]

gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	100   	-50999.6	49990.4    	0.871225   	-100000    


2024-10-04 00:39:51.056347: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-10-04 00:39:51.056462: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-10-04 00:39:51.056483: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-10-04 00:39:51.056556: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-10-04 00:39:51.056589: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


1  	200   	-9999.22	30000.3    	0.871225   	-100000    
2  	200   	-5999.18	23748.9    	0.871271   	-100000    
3  	200   	-5999.18	23748.9    	0.87133    	-100000    
4  	200   	-5999.18	23748.9    	0.87133    	-100000    
5  	200   	-2999.16	17058.9    	0.871482   	-100000    
6  	200   	-6999.19	25514.9    	0.871518   	-100000    
7  	200   	-5999.18	23748.9    	0.871733   	-100000    
8  	200   	-1999.15	14000.1    	0.871733   	-100000    
9  	200   	-2999.16	17058.9    	0.871422   	-100000    
10 	200   	-5999.18	23748.9    	0.871934   	-100000    


GAFeatureSelectionCV(estimator=LGBMClassifier(force_col_wise=True, n_jobs=-1,
                                              random_state=69, verbose=0),
                     generations=10, max_features=320, n_jobs=-1,
                     population_size=100, scoring='roc_auc')

In [25]:
selected_features_mask = selector.support_

selected_features = X_train.columns[selected_features_mask]
num_features_selected = len(selected_features)
print(f"Number of features selected: {num_features_selected}")
print("Selected Features:")
print("selected features:", selected_features)

Number of features selected: 300
Selected Features:
selected features: Index(['glucose_d1_value_range', 'age', 'glucose_d1_min_icu_id_mean', 'arf_apache', 'age_counts', 'glucose_d1_max_icu_id_norm_rank', 'glucose_apache_d1_max_ratio', 'd1_glucose_max', 'diabete_risk', 'spo2_h1_max_icu_id_mean',
       ...
       'spo2_d1_min_icu_id_rank', 'spo2_d1_min_icu_id_norm_rank', 'sysbp_invasive_h1_max_apache_3j_diagnosis_norm_std', 'sysbp_invasive_h1_max_apache_3j_diagnosis_diff', 'hco3_d1_min_profile_norm_rank', 'spo2_d1_min_icu_id_std', 'pao2fio2ratio_h1_min_icu_id_skew', 'glucose_h1_min_profile_skew', 'hemaglobin_d1_max_apache_3j_diagnosis_norm_std', 'hemaglobin_d1_max_apache_3j_diagnosis_std'], dtype='object', length=300)


In [26]:
X_train_selected = selector.transform(X_train)
X_test_selected = selector.transform(X_test)

model.fit(X_train_selected, y_train)

# Predict and evaluate
y_pred = model.predict(X_test_selected)
y_pred_proba = model.predict_proba(X_test_selected)[:, 1] 

/Users/sakshamjain/tensorflow69/env/lib/python3.8/site-packages/sklearn/base.py:458: UserWarning: X has feature names, but GAFeatureSelectionCV was fitted without feature names
  warnings.warn(
/Users/sakshamjain/tensorflow69/env/lib/python3.8/site-packages/sklearn/base.py:458: UserWarning: X has feature names, but GAFeatureSelectionCV was fitted without feature names
  warnings.warn(


In [27]:
# Classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# AUC-ROC Score
auc_score = roc_auc_score(y_test, y_pred_proba)
print("AUC-ROC Score:", auc_score)

Classification Report:
              precision    recall  f1-score   support

         0.0       0.87      0.94      0.90     30574
         1.0       0.69      0.49      0.58      8474

    accuracy                           0.84     39048
   macro avg       0.78      0.72      0.74     39048
weighted avg       0.83      0.84      0.83     39048

AUC-ROC Score: 0.870725875873591


In [28]:
X_train_selected_df = pd.DataFrame(X_train_selected, columns=selected_features)

# Save the DataFrame as a CSV file
X_train_selected_df.to_csv('X_train_GA.csv', index=False)

# You can also save the test set if needed
X_test_selected_df = pd.DataFrame(X_test_selected, columns=selected_features)
X_test_selected_df.to_csv('X_test_GA.csv', index=False)

In [29]:
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from lightgbm import early_stopping, log_evaluation

#Assuming X_train, y_train, X_test, y_test are already defined

# Define parameter sets
lparams = {}

lparams[0] = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'learning_rate': 0.01,
    'num_leaves': 200,
    'max_bin': 500,
    'min_child_weight': 0.035,
    'subsample': 0.45,
    'colsample_bytree': 0.3,
    'min_data_in_leaf': 150,
    'max_depth': -1,
    'reg_alpha': 0.4,
    'reg_lambda': 0.7,
    'verbose': 1,
    'random_state': 0,  # Combining seed and bagging_seed for reproducibility
    'n_jobs': -1,
    'n_estimators': 30000
}

# Initialize models
model_0 = lgb.LGBMClassifier(**lparams[0])

In [30]:
# Fit models with early stopping
model_0.fit(
    X_train_selected, y_train,
    eval_set=[(X_test_selected, y_test)],
    eval_metric='auc',
    callbacks=[
        early_stopping(stopping_rounds=300),
        log_evaluation(period=1)  
    ]
)

preds_0 = model_0.predict_proba(X_test_selected)[:, 1]
auc_0 = roc_auc_score(y_test, preds_0)
print(f"Model 0 AUC: {auc_0:.4f}")

[LightGBM] [Warning] min_data_in_leaf is set=150, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=150
[LightGBM] [Warning] min_data_in_leaf is set=150, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=150
[LightGBM] [Info] Number of positive: 19677, number of negative: 71432
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.032759 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 126228
[LightGBM] [Info] Number of data points in the train set: 91109, number of used features: 300
[LightGBM] [Warning] min_data_in_leaf is set=150, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=150
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.215972 -> initscore=-1.289296
[LightGBM] [Info] Start training from score -1.289296
[1]	valid_0's auc: 0.813614
Training until validation scores don't improve for 300 rounds
[2]	valid_0's auc: 0.82618
[3]	valid_0's 

In [31]:
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from lightgbm import early_stopping, log_evaluation

# Assuming xtrain, ytrain, X_test, ytest are already defined

# Define parameter sets
lparams = {}

lparams[0] = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'learning_rate': 0.01,
    'num_leaves': 200,
    'max_bin': 500,
    'min_child_weight': 0.035,
    'subsample': 0.45,
    'colsample_bytree': 0.3,
    'min_data_in_leaf': 150,
    'max_depth': -1,
    'reg_alpha': 0.4,
    'reg_lambda': 0.7,
    'verbose': 1,
    'random_state': 0,  # Combining seed and bagging_seed for reproducibility
    'n_jobs': -1,
    'n_estimators': 30000
}

lparams[1] = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'learning_rate': 0.01,
    'n_estimators': 30000,
    'subsample': 1,
    'colsample_bytree': 0.225,
    'max_depth': -1,
    'reg_alpha': 1,
    'reg_lambda': 1,
    'verbose': 1,
    'random_state': 1,  # Combining seed and bagging_seed for reproducibility
    'n_jobs': -1
}

lparams[2] = {
        'boosting_type': 'dart',
        'objective': 'binary',
        'metric': 'auc',
        'learning_rate': 0.01,
        'subsample': 1,
        'colsample_bytree': 0.1,
        'reg_alpha': 3,
        'reg_lambda': 1,
        'scale_pos_weight': 1,
        'n_estimators': 14000,
        'silent': -1,
        'verbose': -1,
        'max_depth': -1,
        'random_state': 0,
        'n_jobs': -1
    }

lparams[3] = {
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'auc',
        'learning_rate': 0.01,
        'subsample': 1,
        'colsample_bytree': 0.1,
        'reg_alpha': 3,
        'reg_lambda': 1,
        'scale_pos_weight': 1,
        'n_estimators': 300000,
        'silent': -1,
        'verbose': -1,
        'max_depth': -1,
        'random_state': 0,
        'n_jobs': -1

    }

lparams[4] = {
    'objective':'binary'
    ,'boosting': 'goss'
    ,'verbosity': -1
    ,'metric': 'AUC'
    ,'learning_rate': .01
    ,'num_leaves': 63
    ,'min_data_in_leaf': 250
    ,'feature_fraction': .3
    ,'extra_trees': True
    ,'top_rate': .8
    ,'other_rate': .1,
    'random_state': 0,
    'n_jobs': -1
}

# Initialize models
model_0 = lgb.LGBMClassifier(**lparams[0])
model_1 = lgb.LGBMClassifier(**lparams[1])
model_2 = lgb.LGBMClassifier(**lparams[2])
model_3 = lgb.LGBMClassifier(**lparams[3])
model_4 = lgb.LGBMClassifier(**lparams[4])


In [32]:
# Fit models with early stopping
model_1.fit(
    X_train_selected, y_train,
    eval_set=[(X_test_selected, y_test)],
    eval_metric='auc',
    callbacks=[
        early_stopping(stopping_rounds=300),
        log_evaluation(period=1)  
    ]
)

preds_0 = model_1.predict_proba(X_test_selected)[:, 1]
auc_0 = roc_auc_score(y_test, preds_0)
print(f"Model 0 AUC: {auc_0:.4f}")

[LightGBM] [Info] Number of positive: 19677, number of negative: 71432
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.033636 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 71976
[LightGBM] [Info] Number of data points in the train set: 91109, number of used features: 300
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.215972 -> initscore=-1.289296
[LightGBM] [Info] Start training from score -1.289296
[1]	valid_0's auc: 0.797332
Training until validation scores don't improve for 300 rounds
[2]	valid_0's auc: 0.814706
[3]	valid_0's auc: 0.817287
[4]	valid_0's auc: 0.818257
[5]	valid_0's auc: 0.818023
[6]	valid_0's auc: 0.819915
[7]	valid_0's auc: 0.821401
[8]	valid_0's auc: 0.821931
[9]	valid_0's auc: 0.824054
[10]	valid_0's auc: 0.82549
[11]	valid_0's auc: 0.82543
[12]	valid_0's auc: 0.826057
[13]	valid_0's auc: 0.826146
[14]	valid_0's auc: 0.827977
[15]	valid_0's auc: 0.828096
[16]	valid_0's auc

In [33]:
# Fit models with early stopping
model_2.fit(
    X_train_selected, y_train,
    eval_set=[(X_test_selected, y_test)],
    eval_metric='auc',
    callbacks=[
        early_stopping(stopping_rounds=300),
        log_evaluation(period=1)  
    ]
)

preds_0 = model_2.predict_proba(X_test_selected)[:, 1]
auc_0 = roc_auc_score(y_test, preds_0)
print(f"Model 0 AUC: {auc_0:.4f}")

[1]	valid_0's auc: 0.690165
[2]	valid_0's auc: 0.795399
[3]	valid_0's auc: 0.802009
[4]	valid_0's auc: 0.807218
[5]	valid_0's auc: 0.812087
[6]	valid_0's auc: 0.814001
[7]	valid_0's auc: 0.815011
[8]	valid_0's auc: 0.81905
[9]	valid_0's auc: 0.820496
[10]	valid_0's auc: 0.821441
[11]	valid_0's auc: 0.821606
[12]	valid_0's auc: 0.82074
[13]	valid_0's auc: 0.820045
[14]	valid_0's auc: 0.82073
[15]	valid_0's auc: 0.821454
[16]	valid_0's auc: 0.821965
[17]	valid_0's auc: 0.821522
[18]	valid_0's auc: 0.822905
[19]	valid_0's auc: 0.822849
[20]	valid_0's auc: 0.822743


/Users/sakshamjain/tensorflow69/env/lib/python3.8/site-packages/lightgbm/callback.py:329: UserWarning: Early stopping is not available in dart mode
  _log_warning("Early stopping is not available in dart mode")


[21]	valid_0's auc: 0.822813
[22]	valid_0's auc: 0.822663
[23]	valid_0's auc: 0.823673
[24]	valid_0's auc: 0.82742
[25]	valid_0's auc: 0.827309
[26]	valid_0's auc: 0.826936
[27]	valid_0's auc: 0.826319
[28]	valid_0's auc: 0.82665
[29]	valid_0's auc: 0.826333
[30]	valid_0's auc: 0.825881
[31]	valid_0's auc: 0.826064
[32]	valid_0's auc: 0.826334
[33]	valid_0's auc: 0.826695
[34]	valid_0's auc: 0.826276
[35]	valid_0's auc: 0.826893
[36]	valid_0's auc: 0.826853
[37]	valid_0's auc: 0.826899
[38]	valid_0's auc: 0.826968
[39]	valid_0's auc: 0.826875
[40]	valid_0's auc: 0.826883
[41]	valid_0's auc: 0.826916
[42]	valid_0's auc: 0.827277
[43]	valid_0's auc: 0.827116
[44]	valid_0's auc: 0.826989
[45]	valid_0's auc: 0.827085
[46]	valid_0's auc: 0.828259
[47]	valid_0's auc: 0.828396
[48]	valid_0's auc: 0.828971
[49]	valid_0's auc: 0.829017
[50]	valid_0's auc: 0.82934
[51]	valid_0's auc: 0.830281
[52]	valid_0's auc: 0.830093
[53]	valid_0's auc: 0.830075
[54]	valid_0's auc: 0.83005
[55]	valid_0's auc

In [34]:
# Fit models with early stopping
model_3.fit(
    X_train_selected, y_train,
    eval_set=[(X_test_selected, y_test)],
    eval_metric='auc',
    callbacks=[
        early_stopping(stopping_rounds=300),
        log_evaluation(period=1)  
    ]
)

preds_0 = model_3.predict_proba(X_test_selected)[:, 1]
auc_0 = roc_auc_score(y_test, preds_0)
print(f"Model 0 AUC: {auc_0:.4f}")

[1]	valid_0's auc: 0.690165
Training until validation scores don't improve for 300 rounds
[2]	valid_0's auc: 0.795399
[3]	valid_0's auc: 0.802009
[4]	valid_0's auc: 0.807218
[5]	valid_0's auc: 0.812087
[6]	valid_0's auc: 0.815152
[7]	valid_0's auc: 0.816545
[8]	valid_0's auc: 0.819309
[9]	valid_0's auc: 0.820362
[10]	valid_0's auc: 0.821131
[11]	valid_0's auc: 0.821289
[12]	valid_0's auc: 0.820563
[13]	valid_0's auc: 0.819907
[14]	valid_0's auc: 0.820655
[15]	valid_0's auc: 0.821102
[16]	valid_0's auc: 0.821961
[17]	valid_0's auc: 0.822291
[18]	valid_0's auc: 0.823464
[19]	valid_0's auc: 0.823537
[20]	valid_0's auc: 0.823346
[21]	valid_0's auc: 0.82351
[22]	valid_0's auc: 0.823408
[23]	valid_0's auc: 0.824083
[24]	valid_0's auc: 0.826577
[25]	valid_0's auc: 0.826191
[26]	valid_0's auc: 0.826031
[27]	valid_0's auc: 0.826452
[28]	valid_0's auc: 0.826883
[29]	valid_0's auc: 0.826877
[30]	valid_0's auc: 0.826617
[31]	valid_0's auc: 0.826725
[32]	valid_0's auc: 0.826829
[33]	valid_0's auc: 

In [35]:
# Fit models with early stopping
model_4.fit(
    X_train_selected, y_train,
    eval_set=[(X_test_selected, y_test)],
    eval_metric='auc',
    callbacks=[
        early_stopping(stopping_rounds=300),
        log_evaluation(period=1)  
    ]
)

preds_0 = model_4.predict_proba(X_test_selected)[:, 1]
auc_0 = roc_auc_score(y_test, preds_0)
print(f"Model 0 AUC: {auc_0:.4f}")

[1]	valid_0's auc: 0.801012
Training until validation scores don't improve for 300 rounds
[2]	valid_0's auc: 0.814275
[3]	valid_0's auc: 0.822908
[4]	valid_0's auc: 0.824822
[5]	valid_0's auc: 0.827758
[6]	valid_0's auc: 0.829216
[7]	valid_0's auc: 0.829151
[8]	valid_0's auc: 0.830057
[9]	valid_0's auc: 0.831088
[10]	valid_0's auc: 0.83094
[11]	valid_0's auc: 0.831619
[12]	valid_0's auc: 0.832354
[13]	valid_0's auc: 0.832442
[14]	valid_0's auc: 0.8326
[15]	valid_0's auc: 0.832983
[16]	valid_0's auc: 0.832918
[17]	valid_0's auc: 0.832991
[18]	valid_0's auc: 0.833442
[19]	valid_0's auc: 0.833093
[20]	valid_0's auc: 0.833352
[21]	valid_0's auc: 0.833805
[22]	valid_0's auc: 0.833807
[23]	valid_0's auc: 0.833952
[24]	valid_0's auc: 0.83418
[25]	valid_0's auc: 0.834278
[26]	valid_0's auc: 0.834373
[27]	valid_0's auc: 0.834472
[28]	valid_0's auc: 0.834711
[29]	valid_0's auc: 0.834786
[30]	valid_0's auc: 0.834857
[31]	valid_0's auc: 0.834853
[32]	valid_0's auc: 0.834966
[33]	valid_0's auc: 0.8

In [37]:
X_test_selected_df

,glucose_d1_value_range,age,glucose_d1_min_icu_id_mean,arf_apache,age_counts,glucose_d1_max_icu_id_norm_rank,glucose_apache_d1_max_ratio,d1_glucose_max,diabete_risk,spo2_h1_max_icu_id_mean,glucose_d1_max_apache_3j_diagnosis_norm_rank,glucose_d1_min_icu_id_std,glucose_h1_max_icu_id_skew,temp_h1_min_icu_id_mean,arterial_ph_h1_min_icu_id_mean,d1_wbc_max_bmi_ratio,arterial_po2_h1_min_apache_3j_diagnosis_mean,icu_id,pre_icu_los_days,glucose_d1_max_apache_3j_diagnosis_diff,glucose_d1_min_apache_3j_diagnosis_mean,spo2_d1_max_icu_id_skew,hemaglobin_d1_max_icu_id_diff,pao2fio2ratio_d1_min_icu_id_skew,glucose_apache,glucose_h1_min_icu_id_skew,glucose_d1_min_apache_3j_diagnosis_norm_rank,d1_diasbp_max_bmi_ratio,glucose_d1_max_icu_id_skew,potassium_d1_max_icu_id_skew,glucose_d1_mean,glucose_d1_h1_mean,apache_3j_diagnosis_split1,hco3_d1_value_range,glucose_d1_min_apache_3j_diagnosis_std,sysbp_invasive_d1_h1_mean,d1_creatinine_min,potassium_d1_min_icu_id_mean,ethnicity_Caucasian,glucose_d1_max_icu_id_mean,temp_d1_min_icu_id_norm_rank,diasbp_noninvasive_h1_max_icu_id_mean,creatinine_d1_h1_mean,arterial_ph_d1_min_icu_id_mean,platelets_d1_min_profile_diff,d1_bun_min,calcium_d1_min_icu_id_norm_rank,platelets_d1_min_icu_id_diff,diasbp_d1_min_icu_id_mean,mbp_invasive_d1_min_icu_id_skew,platelets_h1_min_icu_id_mean,glucose_d1_min_icu_id_norm_std,d1_lactate_max_bmi_ratio,wbc_d1_max_apache_3j_diagnosis_norm_rank,h1_diasbp_min_weight_ratio,d1_resprate_min_bmi_ratio,sysbp_noninvasive_d1_h1_mean,spo2_d1_max_icu_id_norm_rank,heartrate_d1_min_icu_id_std,weight_age,arterial_po2_h1_max_icu_id_skew,diasbp_noninvasive_h1_min_icu_id_skew,hematocrit_d1_value_range,glucose_d1_min_icu_id_diff,bun_d1_h1_mean,mbp_d1_min_apache_3j_diagnosis_std,sysbp_invasive_d1_min_icu_id_norm_rank,glucose_d1_max_apache_3j_diagnosis_norm_std,gcs_unable_apache_indicator,hematocrit_d1_max_icu_id_norm_std,hematocrit_d1_min_icu_id_skew,sysbp_noninvasive_d1_min_profile_diff,hematocrit_h1_max_apache_3j_diagnosis_std,sysbp_noninvasive_d1_mean,sysbp_noninvasive_d1_min_icu_id_mean,creatinine_h1_min_apache_3j_diagnosis_std,sysbp_invasive_d1_max_apache_3j_diagnosis_skew,creatinine_d1_mean,arterial_ph_d1_min_profile_skew,arterial_pco2_d1_max_profile_rank,calcium_d1_min_apache_3j_diagnosis_norm_rank,calcium_d1_min_profile_norm_std,d1_creatinine_max_bmi_ratio,resprate_h1_max_icu_id_norm_std,mbp_d1_max_icu_id_skew,sysbp_noninvasive_h1_max_apache_3j_diagnosis_norm_rank,platelets_d1_min_icu_id_rank,bun_d1_min_icu_id_std,mbp_invasive_d1_max_icu_id_skew,bun_d1_min_profile_mean,bun_d1_max_apache_3j_diagnosis_mean,bun_h1_max_icu_id_norm_std,diasbp_invasive_d1_max_profile_rank,d1_arterial_po2_max_indicator,d1_resprate_div_sysbp_min,pao2fio2ratio_d1_min_apache_3j_diagnosis_norm_rank,temp_d1_value_range,diasbp_invasive_d1_min_icu_id_std,diasbp_invasive_d1_max_icu_id_std,platelets_d1_min_apache_3j_diagnosis_norm_rank,platelets_h1_max_icu_id_skew,platelets_h1_min_profile_mean,heartrate_d1_min_apache_3j_diagnosis_norm_rank,mbp_noninvasive_h1_max_icu_id_mean,sodium_d1_max_apache_3j_diagnosis_mean,platelets_d1_max_profile_norm_rank,glucose_d1_min_apache_3j_diagnosis_norm_std,albumin_h1_max_icu_id_skew,hco3_d1_min_icu_id_rank,hco3_d1_max_icu_id_skew,d1_glucose_min_weight_ratio,glucose_apache_h1_min_ratio,sysbp_invasive_h1_max_apache_3j_diagnosis_rank,heartrate_d1_value_range,h1_diasbp_noninvasive_max_bmi_ratio,potassium_h1_min_apache_3j_diagnosis_norm_std,gcs_unable_apache,arterial_po2_h1_min_profile_norm_std,diasbp_invasive_h1_min_icu_id_mean,arterial_pco2_h1_max_apache_3j_diagnosis_mean,wbc_h1_max_profile_std,hemaglobin_d1_max_apache_3j_diagnosis_norm_rank,hco3_d1_max_profile_mean,wbc_h1_max_icu_id_mean,creatinine_d1_max_apache_3j_diagnosis_mean,diasbp_invasive_d1_max_icu_id_norm_rank,diasbp_invasive_d1_max_icu_id_skew,creatinine_apache_h1_max_ratio,creatinine_h1_min_icu_id_skew,arterial_po2_d1_min_apache_3j_diagnosis_skew,hco3_d1_min_apache_3j_diagnosis_norm_rank,creatinine_d1_max_profile_norm_std,d1_platel

In [38]:
# Classification report

model.fit(X_train_selected_df, y_train)

# Predict and evaluate
y_pred = model.predict(X_test_selected_df)
y_pred_proba = model.predict_proba(X_test_selected_df)[:, 1] 

print("Classification Report:")
print(classification_report(y_test, y_pred))

# AUC-ROC Score
auc_score = roc_auc_score(y_test, y_pred_proba)
print("AUC-ROC Score:", auc_score)

Classification Report:
              precision    recall  f1-score   support

         0.0       0.87      0.94      0.90     30574
         1.0       0.69      0.49      0.58      8474

    accuracy                           0.84     39048
   macro avg       0.78      0.72      0.74     39048
weighted avg       0.83      0.84      0.83     39048

AUC-ROC Score: 0.870725875873591


: 